# Omdena  - Milan Chapter Agrifoods
## AI for Sustainable agri-food systems: use of Satellite Imagery
### Exploratory analysis of grapes and olives production in Italy 2006-2022
#### Author: Maria Fisher 


The main objective of this study is to have gather information about crop production in Italy for the period of 2006-2022. 

Crop dataset used in this study was downloaded from the Italian National Institute of Statistics (Istat).



In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import os
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns 
import scipy 
import sklearn
import geopandas as gpd
import pgeocode
import folium
import sys
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [ ]:
grapes_olives = pd.read_csv('./Italy_crop_data/grapes_olives.csv',skipinitialspace=True)
grapes_olives.head()

## Pre-processing dataset 

In [ ]:
# Drop Columns
grapes_olives = grapes_olives.drop(columns =['ITTER107','TIPO_DATO5','AGRI_MADRE', 'TIME', 'Flag Codes','Flags' ])
grapes_olives

In [ ]:
# Rename Columns
grapes_olives = grapes_olives.rename(columns = {'Select time':'Year', 'Type of crop':'Type_crop', 'Data type':'Data_type', 'Territory':'City'})


In [ ]:
def show_info(grapes_olives):
    print('DATASET SHAPE: ', grapes_olives.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(grapes_olives.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(grapes_olives.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(grapes_olives.isnull().sum())
show_info(grapes_olives)

## Cities in Italy producing grapes_olives

In [ ]:
# Check cities names
grapes_olives.City.unique()


In [ ]:
# Select only data for total production - quintals
grapes_olives_total_prod = grapes_olives[grapes_olives['Data_type'] == 'total production - quintals ']
grapes_olives_total_prod.head()

In [ ]:
# Transform values from quintals to tonnes
grapes_olives_total_prod['Value'] = grapes_olives_total_prod['Value']/10
grapes_olives_total_prod

In [ ]:
# Rename column value to production tonnes
grapes_olives_total_prod = grapes_olives_total_prod.rename(columns = {'Value':'Production_tonnes'})
grapes_olives_total_prod

In [ ]:
# drop column data type
grapes_olives_total_prod =grapes_olives_total_prod.drop(columns =['Data_type' ])
grapes_olives_total_prod

In [ ]:
# Import climate dataset
climate_mean = pd.read_csv('climate_data/climate2006-2021_mean.csv')
climate_mean.head()

In [ ]:
# Check cities names
climate_mean.City.unique()

In [ ]:
# Select cities names that correlate with crop data 

climate_mean = climate_mean.apply(lambda row: row[climate_mean['City'].isin(['Agrigento', 'Alessandria', 'Ancona', 'Arezzo', 'Ascoli Piceno',
                                                                             'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
                                                                             'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
                                                                             'Brescia', 'Brindisi', 'Cagliari', 'Caltanissetta', 'Campobasso',
                                                                             'Carbonia-Iglesias', 'Caserta', 'Catania', 'Catanzaro', 'Chieti',
                                                                             'Como', 'Cosenza', 'Cremona', 'Crotone', 'Cuneo', 'Enna', 'Fermo',
                                                                             'Ferrara', 'Firenze', 'Foggia', 'Forlì-Cesena', 'Frosinone',
                                                                             'Genova', 'Gorizia', 'Grosseto', 'Imperia', 'Isernia', "L'Aquila",
                                                                             'La Spezia', 'Latina', 'Lecce', 'Lecco', 'Livorno', 'Lodi',
                                                                             'Lucca', 'Macerata', 'Mantova', 'Massa-Carrara', 'Matera',
                                                                             'Medio Campidano', 'Messina', 'Milano', 'Modena',
                                                                             'Monza e della Brianza', 'Napoli', 'Novara', 'Nuoro', 'Ogliastra',
                                                                             'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo', 'Parma', 'Pavia',
                                                                             'Perugia', 'Pesaro e Urbino', 'Pescara', 'Piacenza', 'Pisa',
                                                                             'Pistoia', 'Pordenone', 'Potenza', 'Prato', 'Ragusa', 'Ravenna',
                                                                             'Reggio di Calabria', "Reggio nell'Emilia", 'Rieti', 'Rimini',
                                                                             'Roma', 'Rovigo', 'Salerno', 'Sassari', 'Savona', 'Siena',
                                                                             'Siracusa', 'Sondrio', 'Sud Sardegna', 'Taranto', 'Teramo',
                                                                             'Terni', 'Torino', 'Trapani', 'Trentino Alto Adige / Südtirol',
                                                                             'Trento', 'Treviso', 'Trieste', 'Udine',
                                                                             "Valle d'Aosta / Vallée d'Aoste", 'Varese', 'Venezia',
                                                                             'Verbano-Cusio-Ossola', 'Vercelli', 'Verona', 'Vibo Valentia',
                                                                             'Vicenza', 'Viterbo'])])

climate_mean.head()

In [ ]:
climate_mean.City.unique()

In [ ]:
# Join both crop and climate datasets 
climate_grapes_olives = pd.merge_ordered(grapes_olives_total_prod, climate_mean,  left_by="City").fillna(0)

In [ ]:
climate_grapes_olives.head()

Import fertilizers data 

In [ ]:
fertilizer = pd.read_csv('crops_climate_fertilizers/fertilizer2006-2021.csv',skipinitialspace=True)
fertilizer.head()

In [ ]:
# Drop irrelevant colmuns 
crop_fertilizer_climate = crop_fertilizer_climate.drop(columns =['Unnamed: 0.1','Unnamed: 0', 'Type_fertilizer'])
crop_fertilizer_climate

Join crop and fertilizers datasets 

In [ ]:
# Join both crop and fertilizers datasets 
crop_fertilizer_climate = pd.merge(climate_grapes_olives , fertilizer, on=['Year', 'City'], how='left').fillna(0)
crop_fertilizer_climate

In [ ]:
# Save crop and fertilizers joined data
crop_fertilizer_climate  =  crop_fertilizer_climate.to_csv('grapes_olives_fertilizer_climate2006-2021.csv', index=False)

In [ ]:
crop_fertilizer_climate = pd.read_csv('grapes_olives_fertilizer_climate2006-2021.csv',skipinitialspace=True)
crop_fertilizer_climate.head()

Import locations lat and lon 

In [ ]:
geo = pd.read_csv('it_locations2.csv',skipinitialspace=True)
geo.head()

In [ ]:
geo.City.unique()

In [ ]:
# Select cities that correlate with crop data 

geo = geo.apply(lambda row: row[geo['City'].isin(['Agrigento', 'Alessandria', 'Ancona', 'Arezzo', 'Ascoli Piceno',
                                                                             'Asti', 'Avellino', 'Bari', 'Barletta-Andria-Trani', 'Belluno',
                                                                             'Benevento', 'Bergamo', 'Biella', 'Bologna', 'Bolzano / Bozen',
                                                                             'Brescia', 'Brindisi', 'Cagliari', 'Caltanissetta', 'Campobasso',
                                                                             'Carbonia-Iglesias', 'Caserta', 'Catania', 'Catanzaro', 'Chieti',
                                                                             'Como', 'Cosenza', 'Cremona', 'Crotone', 'Cuneo', 'Enna', 'Fermo',
                                                                             'Ferrara', 'Firenze', 'Foggia', 'Forlì-Cesena', 'Frosinone',
                                                                             'Genova', 'Gorizia', 'Grosseto', 'Imperia', 'Isernia', "L'Aquila",
                                                                             'La Spezia', 'Latina', 'Lecce', 'Lecco', 'Livorno', 'Lodi',
                                                                             'Lucca', 'Macerata', 'Mantova', 'Massa-Carrara', 'Matera',
                                                                             'Medio Campidano', 'Messina', 'Milano', 'Modena',
                                                                             'Monza e della Brianza', 'Napoli', 'Novara', 'Nuoro', 'Ogliastra',
                                                                             'Olbia-Tempio', 'Oristano', 'Padova', 'Palermo', 'Parma', 'Pavia',
                                                                             'Perugia', 'Pesaro e Urbino', 'Pescara', 'Piacenza', 'Pisa',
                                                                             'Pistoia', 'Pordenone', 'Potenza', 'Prato', 'Ragusa', 'Ravenna',
                                                                             'Reggio di Calabria', "Reggio nell'Emilia", 'Rieti', 'Rimini',
                                                                             'Roma', 'Rovigo', 'Salerno', 'Sassari', 'Savona', 'Siena',
                                                                             'Siracusa', 'Sondrio', 'Sud Sardegna', 'Taranto', 'Teramo',
                                                                             'Terni', 'Torino', 'Trapani', 'Trentino Alto Adige / Südtirol',
                                                                             'Trento', 'Treviso', 'Trieste', 'Udine',
                                                                             "Valle d'Aosta / Vallée d'Aoste", 'Varese', 'Venezia',
                                                                             'Verbano-Cusio-Ossola', 'Vercelli', 'Verona', 'Vibo Valentia',
                                                                             'Vicenza', 'Viterbo'])])
geo.head()

In [ ]:
geo.City.unique()

In [ ]:
# Join both crop and locations datasets 
crop_fertilizer_climate = pd.merge_ordered(crop_fertilizer_climate, geo,  left_by="City").fillna(0)

In [ ]:
crop_fertilizer_climate

In [ ]:
# Save final dataset
crop_fertilizer_climate  =  crop_fertilizer_climate.to_csv('grapes-olives_climate2006-2021.csv', index=False)